## Q1 Ad Campaigns
We have hundreds of millions of active users on Twitter every day. 
Some of them create an advertising account (on ads.twitter.com). Each advertising account has a unique account_id. Once a user has created an advertising account, the user can start creating ad campaigns on Twitter (e.g. to get new followers or promote tweets).
Each campaign the user creates is identified by a unique campaign_id and has one campaign type. 
Below are two related tables/datasets that we’d like you to extract insights from. 

Table 1: Campaigns (This table provides the list of Campaigns created by an Account)

|Account_ID		| Campaign_ID 		| Campaign_Type|
| ------------- |:-------------:| :-----:|
|1			| 101				| Promoted Trend|
|1			| 102				| Promoted Account|
|1			| 103				| Promoted Trend|
|2			| 201				| Promoted Tweet|

Table 2: Spend (This table provides the daily spend per campaign)

|Campaign_ID| Date      		| Spend_USD| 
| ------------- |:-------------:| -----:|
|101			| 2020-01-01		| 200|
|102			| 2020-01-02		| 150|
|101			| 2020-01-02		| 500|
|201			| 2020-01-02		| 250|
|103			| 2020-01-03		| 300|

We call "activation day" the first day where an advertiser account starts spending.

> Q: For each advertiser account, please return the campaign types of the campaigns launched on the account's activation day, together with the total revenue of those campaign types.
The total revenue refers to the entire revenue generated up to the most recent available day (i.e. today).
The output should have the following columns: account id, campaign_type, total_revenue.


In [ ]:
# SQL Query
'''
SELECT
Account_ID as account_id,
Campaign_Type as campaign_type,
SUM(Spend_USD) as total_revenue
FROM
(
	SELECT
	Account_ID,
	c.Campaign_ID,
	Campaign_Type,
	Spend_USD,
	RANK() OVER (PARTITION BY Account_ID ORDER BY Date ASC) as rank
	FROM
	Campaigns c
	LEFT JOIN
	Spend s 
	ON c.Campaign_ID = s.Campaign_ID
) as activation_date
WHERE rank = 1
GROUP BY Account_ID, Campaign_Type;
'''

In [1]:
DATA_CAMPAIGNS = '''
1			| 101				| Promoted Trend
1			| 102				| Promoted Account
1			| 103				| Promoted Trend
2			| 201				| Promoted Tweet
'''

DATA_SPEND = '''
101			| 2020-01-01		| 200
102			| 2020-01-02		| 150
101			| 2020-01-02		| 500
201			| 2020-01-02		| 250
103			| 2020-01-03		| 300
'''

# clean the data and store them in a list of tuples
data_campaigns = [tuple([item.strip() for item in row.split('|')]) for row in DATA_CAMPAIGNS.split('\n') if len(row) > 0]
data_spend = [tuple([item.strip() for item in row.split('|')]) for row in DATA_SPEND.split('\n') if len(row) > 0]

# use campaign data to left join spend data
data_merged = []
for cam in data_campaigns:
    for spd in data_spend:
        if cam[1] == spd[0]:
            data_merged.append(tuple(list(cam)+list(spd[1:])))

# find unique users
users = set([i[0] for i in data_merged])

# loop every user to find their result and store the result in an empty list
res = []
for user in users:
    # partition the dataset by user
    data_user = [data for data in data_merged if data[0] == user]
    # find the activation date for every user
    activation_date = min([data[3] for data in data_user])
    # find the campaign types for every user
    campaign_types = set([data[2] for data in data_user if data[3] == activation_date])
    for campaign_type in campaign_types:
        # change the Spend_USD string into float rather than int, in case there is any float numbers
        res.append(tuple([user, campaign_type, sum([float(data[4]) for data in data_user if data[2] == campaign_type])]))

# sort by account_id ascendingly
res.sort(key=lambda x: x[0])

print('account_id  campaign_type   total_revenue')
for item in res:
    print('{:>6d}\t{:>18s}\t{:>1.1f}'.format(int(item[0]), item[1], item[2]))

account_id  campaign_type   total_revenue
     1	    Promoted Trend	1000.0
     2	    Promoted Tweet	250.0


## Q2 Tweets
We store tweets, users and logins information in the following three tables:

Users

id 	| country 	| signup_date
 ------------- |:-------------:| -----:
1 	| Japan 	| 2019-07-01
2 	| Morocco 	| 2020-09-22


Tweets

id 	| user_id 	| text| created_at
 --- |:-------------:| :-----:| :---:
733 	| 1 		| I love spanish #food!	| 2020-01-01 19:10:12
734 	| 2 		| Today is #raining		| 2020-01-02 20:11:19

Logins

user_id 	| date 				| client
 ------------- |:-------------:| :-----
1 		| 2019-07-01 10:10:09		| mobile-ios
2 		| 2019-09-22 11:25:43		| mobile-android
1 		| 2019-10-30 22:09:21		| web

> Q: Please calculate the top 5 countries with the highest percentage of users who used the hashtag “#food” in at least one tweet.
We are interested only in users who have logged in from a mobile device at least once.
The users who have never logged in from a mobile device should be ignored.


> Answer Notes

Notes: In order to avoid ambiguity, I have considered two cases for this question.

Case1: The "highest percentage" refers to countries that account for the highest percentages in all mobile-login users with food as their hashtag. For example, if Japan has 8 users and Morocco has 2 users, the two countries would take up 80% and 20% respectively. 
The business value of this case is that we could know which countries have the most users tagging “food”.

Case2：The "highest percentage" refers to the percentage calculated by dividing (food-tagged users and mobile-login users) with (mobile-login users). For example, if Japan has 10 mobile login users and among them 86 have tagged “food”, then the percentage of Japan is 60%. Compare the percentage of each country and retrieve the top 5 ones. 
The business meaning of this case is that we could know which countries have the highest percentages of foodies.

Both cases have the same beginning step that the Users table left joins the Tweets table, with whether user IDs are mobile-login users in the Logins table as the screening criterion. Joined_table is created as the source table for the steps below.

For case1, keep data with “food” as its hashtag and count the number of distinct users for each country group by country. Then use a window function to calculate the percentage each country takes on and retrieve the top 5 ones.

For csae2, derive two tables for the numerator(up) and the denominator(down). Since the denominator would not be empty if a country has mobile-login users and does not have food-tagged users, use the denominator part to left join the numerator by country name and calculate the percentage. Sort the result by percentage descendingly and retrieve the top 5 ones.

In [ ]:
# SQL Query

### CASE1 ###
'''
WITH joined_table AS 
(SELECT 
u.id as user_id,
country,
t.id as tweet_id,
text
FROM
Users u
LEFT JOIN
Tweets t
ON u.id = t.user_id
WHERE u.id IN (SELECT DISTINCT user_id FROM Logins WHERE client LIKE "%mobile%")
)


SELECT
country,
ROUND(100 * user_num / SUM(user_num) OVER (), 2) as percentage
FROM
(
	SELECT 
	country,
	COUNT(DISTINCT user_id) as user_num
	FROM
	joined_table
	WHERE text LIKE "%#food%"
	GROUP BY country
) as mobile_food
ORDER BY user_num / SUM(user_num) OVER () DESC 
LIMIT 5;
'''

### CASE2 ###
'''
WITH joined_table AS 
(SELECT 
u.id as user_id,
country,
t.id as tweet_id,
text
FROM
Users u
LEFT JOIN
Tweets t
ON u.id = t.user_id
WHERE u.id IN (SELECT DISTINCT user_id FROM Logins WHERE client LIKE "%mobile%")
)

SELECT 
down.country, 
ROUND(IFNULL(target_users, 0) / all_users * 100, 2) as percentage 
FROM
(
	SELECT 
	country, 
	COUNT(DISTINCT user_id) as all_users 
	FROM joined_table 
	GROUP BY country
) as down
LEFT JOIN
(
	SELECT country, 
	COUNT(DISTINCT user_id) as target_users 
	FROM joined_table 
	WHERE LOWER(text) LIKE "%#food%" 
	GROUP BY country
) as up 
on down.country = up.country
ORDER BY target_users/all_users DESC LIMIT 5;
'''

In [2]:
USERS = {
    1: {"country": "Japan", "signup_date": "2019-07-01"},
    2: {"country": "Morocco", "signup_date": "2020-09-22"},
}

TWEETS = {
    733: {"user_id": 1, "text": "I love spanish #food!", "created_at": "2020-01-01 19:10:12"},
    734: {"user_id": 2, "text": "Today is #raining", "created_at": "2020-01-02 20:11:19"},
}

LOGINS = [
    {"user_id": 1, "date": "2019-07-01 10:10:09", "client": "mobile-ios"},
    {"user_id": 2, "date": "2019-09-22 11:25:43", "client": "mobile-android"},
    {"user_id": 1, "date": "2019-10-30 22:09:21", "client": "web"},
]

# first find out all user ids that have logged in from a mobile device at least once,
# we define mobile_id as a set because it should not contain duplicate user ids.
seen_id = []
mobile_id_set = set()
for login_info in LOGINS:
    user_id = login_info["user_id"]
    if user_id in seen_id:
        continue
    seen_id.append(user_id)
    if "mobile" in login_info["client"]:
        mobile_id_set.add(user_id)

food_id_set = set()

for tweet in TWEETS.values():
    if tweet["user_id"] not in mobile_id_set:
        continue
    if '#food' in tweet["text"]:
        food_id_set.add(tweet["user_id"])

### CASE1 ###
country_num = {}
# country_num is a dictionary with country as its keys and percentage a country takes as its values
for user_id in food_id_set:
    cty = USERS[user_id]["country"]
    if cty in country_num:
        country_num[cty] += 1
    else:
        country_num[cty] = 1
total = sum(country_num.values())
for cty in country_num.keys():
    country_num[cty] /= total
country_ratio_sorted = sorted(country_num.items(), key=lambda x: (-x[1], x[0]))
print('CASE1:')
print('Ranking      Country      Percentage')
for i, item in enumerate(country_ratio_sorted[:5], 1):
    print('{:>6d}\t{:>12s}\t{:>10.2%}'.format(i, item[0], item[1]))

### CASE2 ### 
country_num = {}
# country_num is a dictionary with country as its key and 
# [the number of target users, the number of non-target users] as its value
for user_id in mobile_id_set:
    cty = USERS[user_id]["country"]
    if user_id in food_id_set:
        if cty in country_num:
            country_num[cty][0] += 1
        else:
            country_num[cty] = [1, 0]
    else:
        if cty in country_num:
            country_num[cty][1] += 1
        else:
            country_num[cty] = [0, 1]

country_ratio = {key: float(value[0]) / sum(value) for key, value in country_num.items()}
# sort by percentage descendingly. if two percentages are the same, sort by country alphabetically
country_ratio_sorted = sorted(country_ratio.items(), key=lambda x: (-x[1], x[0]))
print('CASE2:')
print('Ranking      Country      Percentage')
for i, item in enumerate(country_ratio_sorted[:5], 1):
    print('{:>6d}\t{:>12s}\t{:>10.2%}'.format(i, item[0], item[1]))

CASE1:
Ranking      Country      Percentage
     1	       Japan	   100.00%
CASE2:
Ranking      Country      Percentage
     1	       Japan	   100.00%
     2	     Morocco	     0.00%
